In [1]:
import numpy as np
import json
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

from datetime import date

In [2]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import FeatureUnion, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
import itertools as it


In [3]:
df = pd.read_hdf("dftest_v4_filtrado_means.h5")

In [4]:
def printMetrics(label_test,label_predicted,model):
    score = accuracy_score(label_test, label_predicted)
    print(score)
    print(confusion_matrix(label_test, label_predicted, labels=model.classes_))
    print(classification_report(label_test, label_predicted))

In [5]:
bandas = ["b1_60","b2_10","b3_10","b4_10","b5_20","b6_20","b7_20","b8_10","b8a_20","b9_60","b10_60","b11_20","b12_20"]
#indices = ["min","max","mean","var","skew","kurt"]
medias  = [f+"_mean" for f in bandas]


In [6]:
class IndexSelector(BaseEstimator, TransformerMixin):
    """Create all possible indexes by using band combination"""

    def __init__(self, key_array):
        self.key = key_array

    
    def fit(self, x, y=None):
        return self

    def transform(self, x):
        new_X = x[self.key]
        
        return new_X

In [7]:
class IndexTransformer(BaseEstimator, TransformerMixin):
    """Create all possible indexes by using band combination"""

    def fit(self, x, y=None):
        return self

    def transform(self, x):
        pairs = list(it.combinations(medias, 2))
        new_X = pd.DataFrame()
        for a, b in pairs:
            new_X["{}{}".format(a, b)] = (x[a] - x[b])/(x[a] + x[b])
        
        return new_X

In [8]:
class SAVIIndex(BaseEstimator, TransformerMixin):
    """Create all possible indexes by using band combination"""

    def fit(self, x, y=None):
        return self

    def transform(self, x):
        b4 = "b4_10_mean"
        b8 = "b8_10_mean"
        l = 0.428
        # (b8 - b4) / (b8 + b4 +L ) * 1.0 + L

        new_X = pd.DataFrame()
        new_X["SAVI"] = (x[b8] - x[b4])/(x[b8] + x[b4] + l) * 1.0 + l
        
        return new_X

In [9]:
class SIPIIndex(BaseEstimator, TransformerMixin):
    """Create all possible indexes by using band combination"""

    def fit(self, x, y=None):
        return self

    def transform(self, x):
        b4 = "b4_10_mean"
        b8 = "b8_10_mean"
        b1 = "b1_60_mean"
        
        # (b8 - b1) / (b8 - b4)

        new_X = pd.DataFrame()
        new_X["SIPI"] = (x[b8] - x[b1])/(x[b8] - x[b4])
        
        return new_X

In [10]:
class PSSRIndex(BaseEstimator, TransformerMixin):
    """Create all possible indexes by using band combination"""

    def fit(self, x, y=None):
        return self

    def transform(self, x):
        b4 = "b4_10_mean"
        b8 = "b8_10_mean"
        
        # b8 / b4

        new_X = pd.DataFrame()
        new_X["PSSR"] = (x[b8] / x[b4])
        
        return new_X

In [11]:
class EVIIndex(BaseEstimator, TransformerMixin):
    """Create all possible indexes by using band combination"""

    def fit(self, x, y=None):
        return self

    def transform(self, x):
        b4 = "b4_10_mean"
        b8 = "b8_10_mean"
        b2 = "b2_10_mean"
        # 2.5 * (B8 - B4) / (B8 +6 * B4 -7.5*B2) + 1.0

        new_X = pd.DataFrame()
        new_X["EVI"] = 2.5 * (x[b8] - x[b4])/(x[b8] + 6. * x[b4] - 7.5 * x[b2])
        
        return new_X

In [12]:
class EVI2Index(BaseEstimator, TransformerMixin):
    """Create all possible indexes by using band combination"""

    def fit(self, x, y=None):
        return self

    def transform(self, x):
        b4 = "b4_10_mean"
        b8 = "b8_10_mean"

        new_X = pd.DataFrame()
        new_X["EVI2"] = 2.4 * (x[b8] - x[b4])/(x[b8] +  x[b4] + 1.0)
        
        return new_X

In [13]:
class ARVIIndex(BaseEstimator, TransformerMixin):
    """Create all possible indexes by using band combination"""

    def fit(self, x, y=None):
        return self

    def transform(self, x):
        b4 = "b4_10_mean"
        b2 = "b2_10_mean"
        b9 = "b9_60_mean"
        y = 0.106
        new_X = pd.DataFrame()
        new_X["ARVI"] = x[b9] - x[b4] - y * (x[b4] - x[b2]) / (x[b9] + x[b4] - y * (x[b4] - x[b2]))
        
        return new_X

In [14]:
FEATURES_BY_NAME= {
        "index": lambda: IndexTransformer(),
        "savi": lambda: SAVIIndex(),
        "sipi": lambda: SIPIIndex(),
        "pssr": lambda: PSSRIndex(),
        "evi": lambda: EVIIndex(),
        "evi2": lambda: EVI2Index(),
        "arvi": lambda: ARVIIndex()
  }

In [15]:
def get_features_by_name(features):
    t_list = [(feat, FEATURES_BY_NAME[feat]()) for feat in features]
    return FeatureUnion(transformer_list=t_list)

In [16]:
def get_pipeline(features):
    pipe = Pipeline(steps=[('features', get_features_by_name(features)),
                           ('model', RandomForestClassifier(min_samples_split=5,
                                                            max_features=0.9,
                                                            max_depth=110,
                                                            n_estimators=1100,
                                                            random_state = 42,
                                                            n_jobs = -1,
                                                           ))
                          ])

    return pipe

# Trainning

In [17]:
df['class'].unique()

array(['soja', 'maiz', 'girasol'], dtype=object)

In [18]:
df_test = df
# Get X_all
X_all = df_test.drop('class', axis=1)
# Get target variable
y_all = df_test['class']

num_test = 0.30
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all,
                                                    test_size=num_test,
                                                    random_state=23)

In [19]:
X_all.shape

(166802, 13)

In [20]:
features = ['index', 'savi', 'sipi', 'pssr', 'evi', 'evi2', 'arvi']
pipe = get_pipeline(features)

In [21]:
features = pipe.named_steps['features']

In [22]:
model = pipe.named_steps['model']

In [23]:
model

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=110, max_features=0.9, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=5,
            min_weight_fraction_leaf=0.0, n_estimators=1100, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [24]:
X_train_t = features.fit_transform(X_train)

In [25]:
X_test_t = features.fit_transform(X_test)

In [26]:
X_train_t.shape

(116761, 84)

In [27]:
import dask_searchcv as dcv
from dask.diagnostics import ProgressBar

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 3)]
# Number of features to consider at every split

max_features = ['auto', 0.9]
# Maximum number of levels in tree

max_depth = [int(x) for x in np.linspace(10, 110, num = 3)]

max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [8,16,32]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
parameters = {'n_estimators': n_estimators,
              'max_features': max_features,
              'max_depth': max_depth,
              'min_samples_split': min_samples_split,
              'min_samples_leaf': min_samples_leaf,
              'bootstrap': bootstrap}

In [28]:
3 * len(n_estimators) * len(max_features) * len(max_depth) * len(min_samples_leaf) * len(min_samples_split) * len(bootstrap)

864

In [ ]:
search = dcv.RandomizedSearchCV(model, parameters, cv=3,n_jobs=3, n_iter=200)

#https://github.com/dask/dask-searchcv/issues/51
with ProgressBar():
    search.fit(X_train_t, y_train)

[                                        ] | 0% Completed |  1hr 41min 30.7s

/home/daniel/miniconda3/envs/venv/lib/python3.6/site-packages/sklearn/ensemble/base.py:154: UserWarning: Loky-backed parallel loops cannot be nested below threads, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)


[                                        ] | 0% Completed |  1hr 41min 53.9s

/home/daniel/miniconda3/envs/venv/lib/python3.6/site-packages/sklearn/ensemble/base.py:154: UserWarning: Loky-backed parallel loops cannot be nested below threads, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)


[                                        ] | 0% Completed |  2hr 14min 59.0s

...

[##                                      ] | 6% Completed | 28hr 32min 35.6s

/home/daniel/miniconda3/envs/venv/lib/python3.6/site-packages/sklearn/ensemble/base.py:154: UserWarning: Loky-backed parallel loops cannot be nested below threads, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)


[##                                      ] | 6% Completed | 28hr 33min 19.3s

/home/daniel/miniconda3/envs/venv/lib/python3.6/site-packages/sklearn/ensemble/base.py:154: UserWarning: Loky-backed parallel loops cannot be nested below threads, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)


[##                                      ] | 6% Completed | 31hr 22min 39.8s

In [ ]:
print(search.best_params_)    
print(search.best_score_)    

predicted = search.predict(x_test_t)


In [ ]:
printMetrics(y_test,predicted,search)